[OCC](https://gisdata-occokc.opendata.arcgis.com/)

[Enverus](https://www.enverus.com/products/enverus-core/)

[S&P Global](https://www.spglobal.com/commodity-insights/en/products-solutions/upstream-midstream-oil-gas)

In [0]:

def make_csv_path(county="kingfisher", vendor=""):
    catalog  = "geodata"
    schema   = "staging"
    volume   = "raw"
    file     = f"{county.lower()}_well_header_{vendor.lower()}.csv"

    csv_path = f"/Volumes/{catalog}/{schema}/{volume}/{file}"
    return csv_path

In [0]:
import requests
import pandas as pd


OCC_URL = "https://gis.occ.ok.gov/server/rest/services/Hosted/RBDMS_WELLS/FeatureServer/220/query"


def _query_arcgis(county: str) -> pd.DataFrame | None:
    params = {
        "where": f"county='{county.upper()}'",
        "outFields": "*",
        "returnGeometry": "false",
        "f": "json",
        "resultOffset": 0,
        "resultRecordCount": 2000,  # 2_000
    }

    rows = []
    while True:
        print(f"Fetching offset {params['resultOffset']} …")
        data = requests.get(OCC_URL, params=params).json()
        if data.get("error"):
            print("Error received from server:", data["error"])
            return None

        features = data.get("features", [])
        if not features:
            break

        rows.extend(f["attributes"] for f in features)
        if len(features) < params["resultRecordCount"]:
            break
        params["resultOffset"] += params["resultRecordCount"]

    return pd.DataFrame.from_records(rows) if rows else None


def get_occ_well_header_by_county(county="kingfisher") -> str | None:
    volume_path = make_csv_path(county=county, vendor="occ")

    df = _query_arcgis(county)

    if df is None or df.empty:
        print(f"⚠ No data found for {county} county.")
        return None

    df.to_csv(volume_path, index=False)

    return f"✅ {len(df)} records written to {volume_path}"


result = get_occ_well_header_by_county(county="kingfisher")
print(result)

In [0]:
%pip install -q enverus-developer-api

from enverus_developer_api import DeveloperAPIv3
import pandas as pd
import json

SECRET_KEY = dbutils.secrets.get(scope="kingfisher_secrets", key="enverus_secret_key")
STATE = "OK"

v3 = DeveloperAPIv3(secret_key=SECRET_KEY)

def get_enverus_well_header_by_county(county="kingfisher") -> str | None:
    volume_path = make_csv_path(county=county, vendor="enverus")

    records = []
    query = v3.query("well-headers", StateProvince=STATE, County=county.upper())

    for i, record in enumerate(query, start=1):
        records.append(record)

    df = pd.DataFrame.from_records(records)

    df.to_csv(volume_path, index=False)
    return f"✅ {len(df)} records written to '{volume_path}"


result = get_enverus_well_header_by_county(county="kingfisher")
print(result)